In [ ]:
#env
#net
#tnet intilaise all above sequentialy

In [ ]:
!pip install torch torchvision transformers


In [23]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification,AutoFeatureExtractor,AdamW
from tqdm import tqdm
import torch
import numpy as np
from transformers import ViTImageProcessor, ViTModel
from PIL import Image

In [24]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    
    print("CUDA is available. Using GPU.")
else:
    
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [25]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')
# inputs = processor(images=image, return_tensors="pt")
feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
# Step 2: Modify classification head for the new number of classes
num_classes = 10  # Update with your new number of classes
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\sushant\anaconda3\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [26]:
# import cifar10 dataset
import torch
from torchvision import datasets, transforms

# Define a transform to normalize the data
# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Download the training set
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Download the test set
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create DataLoader for training set
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# Create DataLoader for test set
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)
trainloader


Files already downloaded and verified
Files already downloaded and verified


In [27]:
# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

c:\Users\sushant\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
# create buffers frames (sample labeled data) which will be used by our agent.
buffer_data = trainloader

In [13]:
import gym
import gym.spaces
from gym.utils import seeding
from gym.envs.registration import EnvSpec
import enum
import numpy as np


In [38]:
class Actions(enum.Enum):
    Airplane = 0
    Automobile = 1
    Bird = 2
    Cat = 3
    Deer = 4
    Dog = 5
    Frog = 6
    Horse = 7
    Ship = 8
    Truck = 9

In [39]:
class state:
   def __init__(self):
      pass
   
   def step(self,action,label):
      assert isinstance(action, Actions)
      reward = 0.0
      done = False
      close = self._cur_close()
      if action == Actions.Buy and not self.have_position:
         self.have_position = True
         self.open_price = close
         reward -= self.commission_perc
      elif action == Actions.Close and self.have_position:
         reward -= self.commission_perc
         done |= self.reset_on_close
         if self.reward_on_close:
               reward += 100.0 * (close / self.open_price - 1.0)
         self.have_position = False
      self.open_price = 0.0
      self._offset += 1
      prev_close = close
      close = self._cur_close()
      done |= self._offset >= self._prices.close.shape[0]-1

      if self.have_position and not self.reward_on_close:
         reward += 100.0 * (close / prev_close - 1.0)

      return self.observation,reward, done
   def update(self, new_observation):
        # Update the state with the new observation
        self.observation = new_observation
   def reset(self,observation):
       self.observation = observation



In [40]:
class CIFAR10(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self,net ,
                 reset_on_close=True, state_1d=False,
                 random_ofs_on_reset=True, reward_on_close=False,
                 volumes=False):
        super(CIFAR10, self).__init__()
        self.observation_space = gym.spaces.Box(
            low=0, high=255,shape=(80, 80, 3), dtype=np.uint8)
        self.net = net
        self.action_space = gym.spaces.Discrete(n=len(Actions))
        self.state = state()


    def reset(self):
        # Reset the environment
        # Return initial observation
        
        return np.zeros((4,), dtype=np.float32)
    

    def update(self, new_observation):
        # Update the state with the new observation
        self.observation = new_observation


    def step(self, action_idx,label):
        # Take a step in the environment based on the given action
        # Return new observation, reward, done, and info
        self.state.update(self.observation_space)
        action = Actions(action_idx)

        reward, done = self.state.step(action,label)
        info = {
            "instrument": self._instrument,
            "offset": self._state._offset
        }
        return reward, done, info
    def set_observation(self,input_frame):
        self.observation_space = input_frame

    def _reset_environment(self):
        # Implement your environment's reset logic here and return the initial observation
        # This is a placeholder, replace it with your own environment's reset logic
        initial_observation = np.random.randint(0, 256, size=(height, width, 3), dtype=np.uint8)
        return initial_observation
        

In [41]:
env =CIFAR10(model)

In [42]:
#initiate Agent with buffer_data

class Agent:
    def __init__(self,buffer,model,optimizer):
        self.experiance = buffer
        self.model = model
        self.optimizer = optimizer
        self.env = env
    
    def _reset(self):
        self.experiance = None

    
    def _pretrain(self,epoch=4):
        for epoch in range(0,epoch):
            model.train()
            trainloader = self.experiance 
            # Perform your training/validation steps here
            
            # Use Python's iter and next functions to obtain a single batch
            # batch_iterator = iter(trainloader)
            # batchs = next(batch_iterator)
            print(len(trainloader))
            count = 0
            for batch in tqdm(trainloader):
                count += 1
                #print(count)#len(batch),batch[0].size(),batch[1].size())
                running_loss = 0.0
            
                inputs, labels = batch[0].to(device), batch[1].to(device)
                #print(inputs.shape)

                    # inputs = feature_extractor(inputs, return_tensors="pt").to(device)
                outputs = self.model(inputs)
                    # print(outputs)
                    # # Depending on the model version, use either logits or last_hidden_state
                logits = outputs.logits if hasattr(outputs, "logits") else outputs.last_hidden_state
                

                # Compute loss
                loss = torch.nn.functional.cross_entropy(logits, labels)
                   
                    # #loss = outputs.loss
                running_loss += loss.item()
            print(running_loss)
            loss.backward()
            optimizer.step() 
            
            optimizer.zero_grad()
        return model, running_loss
    
    
    def _play_testset_episode(self,testloader):
        #model.eval()
        correct_predictions = 0
        total_samples = 0
        with torch.no_grad():  # No need to compute gradients during testing
            batch_iterator = iter(testloader)
            batch = next(batch_iterator)
            for batch in batch_iterator:
                inputs, labels = batch[0].to(device), batch[1].to(device)
                # Forward pass through the model
                
                outputs = model(inputs)

                # Assuming 'logits' is the key returned by your model
                logits = outputs.logits if hasattr(outputs, "logits") else outputs.last_hidden_state

                # Get the predicted class (index with the maximum probability)
                _, predicted = torch.max(logits, 1)
                print(predicted)
                # Update accuracy statistics
                correct_predictions += (predicted == labels).sum().item()
                total_samples += labels.size(0)
        return total_samples
    def play_episode(self,action,epsilon):
        done_reward = None

        if np.random.random() < epsilon:
            action = env.action_space.sample()
        
        label = "Bird"
        # do step in the environment
        new_state, reward, is_done, _ = self.env.step(action,label)
        self.total_reward += reward

        exp = Experience(self.state, action, reward,
                         is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward
    @torch.no_grad()
    def for_current_episode(self,model,path,epsilon):
        # Step 2: Preprocess the image
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

        image_path = path
        image = Image.open(image_path)
        image_tensor = transform(image)
        # image_tensor = (image_tensor - image_tensor.min()) / (image_tensor.max() - image_tensor.min())  # Normalize to [0, 1]

        # Step 3: Forward pass
        inputs = image_tensor.unsqueeze(0).to(device)  # Add batch dimension
        self.env.set_observation(inputs)
        outputs = model(inputs)

        # Step 4: Post-process predictions
        # Assuming 'logits' is the key returned by your model
        logits = outputs.logits if hasattr(outputs, "logits") else outputs.last_hidden_state
        probabilities = torch.nn.functional.softmax(logits, dim=1)

        # Get the predicted class
        predicted_class = torch.argmax(probabilities, dim=1).item()
        reward = self.play_episode(predicted_class,epsilon)

        return reward,logits
    



In [63]:
agent = Agent(trainloader,model,optimizer)

In [ ]:
model, running_loss=agent._pretrain(epoch=1)

In [68]:
nxt_model = model

In [71]:
def custom_loss(logits,logits_nextstate,gamma,rewards):
    expected_value = logits_nextstate * gamma + rewards
    #print(expected_value)
    return torch.nn.MSELoss()(logits,expected_value)

In [131]:
import numpy as np
np.random.seed(7)

In [1]:
limit = 0.26

running_loss=0
for i in range(0,60):
    model.train()
    optimizer.zero_grad()
    result_s,logits=agent.for_current_episode(model,'./0000.jpg')
    imgclass=0

    result_nextstate,logits_nextstate = agent.for_next_episode(nxt_model,'./0002.jpg')
    nextstateclass = 2
    reward = np.random.rand()
    
    # logits.requires_grad = True
    # logits_nextstate.requires_grad = True
    
    print(reward) 
    loss = custom_loss(logits, logits,0.9,reward)
    loss.requires_grad = True
    loss = custom_loss(logits, logits_nextstate,0.9,reward)
    loss.requires_grad = True
    print(loss)
    running_loss += loss.item()
        
    loss.backward()

    optimizer.step()
    running_loss += loss.item()
    print(f"running loss {running_loss}")
    
    #     if result_s is imgclass and result_nextstate is nextstateclass:
    #         reward = reward+0.75
    #     else:

        
        
    

       
        

NameError: name 'model' is not defined

Now Connect ENV with Agent to 
To do that we need to create envorainment 

In [ ]:
!pip install ptan gym

In [5]:
# Compute Agent loss within two model

